# Introduction

This notebook explains how to clean cosmic void catalogues to extract cosmological constraints from void statistics. 

The only required inputs are the positions of the void centres and of the tracers used to detect the voids (and possibly their masses). 

With this notebook you will remove spurious voids, rescale their radii to a specific density threshold, and reject voids in case of overlaps.

# Set the inputs

Import the CBL functions and system modules:

In [3]:
import CosmoBolognaLib as cbl                                 
from CosmoBolognaLib import ErrorCBL

import time
import sys
import os

To show the standard output stream of the CBL functions (that are implemented in C++ and converted to python via SWIG) on notebook cells (instead of on terminal) the package wurlitzer has to be used (see https://github.com/minrk/wurlitzer)

In [4]:
%load_ext wurlitzer

The code reads the paramters from a file:

In [6]:
filename = "../input/parameters.ini"
print "Loading parameters from", filename
param = cbl.ReadParameters(filename)

Loading parameters from ../input/parameters.ini


The coordinate type are selected (attention: observed coordinates are still not implemented!):

In [7]:
if param.findBool('comovingCoordinates') :
  coordinates = cbl.CoordinateType__comoving_
else :
  coordinates = cbl.CoordinateType__observed_

A cosmological model is defined, using the cosmological parameters provided in the parameter file (this will be used to compute the growth factor)

In [8]:
cosm = cbl.Cosmology(param.findDouble('OmM'),
                     param.findDouble('Omb'),
                     param.findDouble('Omn'),
                     param.findDouble('massless'),
                     param.findInt('massive'),
                     param.findDouble('OmL'),
                     param.findDouble('Omr'),
                     param.findDouble('hh'),
                     param.findDouble('As'),
                     param.findDouble('pivot'),
                     param.findDouble('ns'),
                     param.findDouble('w0'),
                     param.findDouble('wa'),
                     param.findDouble('fNL'),
                     param.findInt('type_NG'),
                     param.findDouble('tau'),
                     param.findString('model'),
                     param.findBool('unit'))

cosm.set_sigma8(param.findDouble('sigma8'))

# Load the input void catalogue
Only the void centre coordinates have to be provided to construct the input void catalogue, while all the other necessary attributes will be computed automatically

In [9]:
cast = []
clmn = []
attrNames = ['X_coord', 'Y_coord', 'Z_coord', 'Radius', 'centralDensity', 'densityContrast']
attrAv = [cbl.Var__X_, cbl.Var__Y_, cbl.Var__Z_, cbl.Var__Radius_, cbl.Var__CentralDensity_,\
          cbl.Var__DensityContrast_]
for ii in range(len(attrNames)) :
  if param.findBool(attrNames[ii]) :
    cast.append(attrAv[ii])
    clmn.append(param.findInt(attrNames[ii]+'_clmn'))
clmn, cast = (list(x) for x in zip(*sorted(zip(clmn, cast))))

attr = cbl.VarCast(cast)

A new void catalogue is created, reading the input file and sorting the attributes according the selected order:

In [10]:
vdcat = cbl.Catalogue(cbl.ObjectType__Void_,
                       coordinates,
                       attr,
                       clmn,
                       [param.findString('inputVoidCatalogue')],
                       param.findInt('vd_comments'))

CBL > I'm reading the catalogue: ../input/vide_void_catalogue.txt


The main properties of the catalogue (volume, density and mean interparticle separation of the sample) are computed using the lenght of the catalogue side. For the current implementation, the catalogue geometry has to be a box. If the boxside is not provided, it will be computed using the maximum separaration between the tracers on the x-axis.

In [11]:
if (param.findDouble('boxside') < 0.) :
  boxside = abs(vdcat.Max(cbl.Var__X_) - vdcat.Min(cbl.Var__X_))
else :
  boxside = param.findDouble('boxside')
vdcat.compute_catalogueProperties(param.findDouble('boxside'))

CBL > Sample volume = 1e+06 (Mpc/h)^3
CBL > Sample density = 0.000221 (Mpc/h)^-3
CBL > Sample mps = 16.54 Mpc/h


# Load the input tracer catalogue

The tracer catalogue can be either a Gadget-2.0 or an ASCII file. Using the parameters 'fact' and 'nSub', it is possible to convert the distance units (e.g. fact = 0.001 for converting kpc/h to Mpc/h) and to sub-sample the objects of the catalogue, respectively. <br> Only for ASCII catalogues, a mass factor and/or a mass cut-off can be applied to the tracer catalogue (obviously only if the tracer mass is provided). The first one can be activated with the parameter 'Munit', that represents the mass units in units of solar masses. The cut-off is applied by selecting a value larger than $0$ for the minimum mass of the catalogue, given by the 'Mmin' parameter.
In the end, the main properties of the catalogue are computed using (once again) the lenght of the boxside.

In [16]:
if param.findBool('Gadget') :
  if not param.findBool('comovingCoordinates') :
    ErrorCBL('Observed coordinates not available for Gadget snapshot.')
  else :
    trcat = cbl.Catalogue (cbl.ObjectType__Halo_,
                           param.findString('inputTracersFile'),
                           param.findBool('swapEndianism'),
                           param.findDouble('fact'),
                           True,
                           param.findDouble('nSub'))
else :
  if param.findBool('comovingCoordinates') :
    tr_cast = []
    tr_clmn = []
    trAttrNames = ['X_coord_tr', 'Y_coord_tr', 'Z_coord_tr', 'Mass']
    trAttrAv = [cbl.Var__X_, cbl.Var__Y_, cbl.Var__Z_, cbl.Var__Mass_]
    for ii in range(len(trAttrNames)) :
      if param.findBool(trAttrNames[ii]) :
        tr_cast.append(trAttrAv[ii])
        tr_clmn.append(param.findInt(trAttrNames[ii]+'_clmn'))        
    tr_clmn, tr_cast = (list(x) for x in zip(*sorted(zip(tr_clmn, tr_cast))))  # orders clmn and cast according to column order 
    tr_attr = cbl.VarCast(tr_cast)
    
    temp = cbl.Catalogue (cbl.ObjectType__Halo_,
                          coordinates,
                          tr_attr,
                          tr_clmn,
                          [param.findString('inputTracersFile')],
                          param.findInt('tr_comments'),
                          param.findDouble('nSub'),
                          param.findDouble('fact'))

    if not param.findBool('Mass') :
      
      trcat = temp
      temp = None
      print "Finished reading input tracers catalogue."
      
    else :
      
      print "Finished reading input tracers catalogue, now applying mass scale factor and/or cut-off ... "

      # scale factor
      if (param.findDouble('Munit') > 0.) :
        for ii in range(temp.nObjects()) :
          mass = temp.mass(ii)*param.findDouble('Munit')
          temp.set_var(ii, cbl.Var__Mass_, mass)

      # mass cut-off
      if (param.findDouble('Mmin') > 0.) :
        trcat = cbl.Catalogue ()
        trcat = temp.sub_catalogue(cbl.Var__Mass_, param.findDouble('Mmin'), temp.Max(cbl.Var__Mass_), False)
      else :
        trcat = temp
        temp = None

      print "\t ... done!"

  # observed coordinates
  else :
    print "Observed coordinates not supported yet..."
    exit(1)

trcat.compute_catalogueProperties(param.findDouble('boxside'))

Finished reading input tracers catalogue.CBL > I'm reading the catalogue: ../input/tracers_catalogue.txt

CBL > Sample volume = 1e+06 (Mpc/h)^3
CBL > Sample density = 0.041706 (Mpc/h)^-3
CBL > Sample mps = 2.88359 Mpc/h


A 3-dimensional chain-mesh for the input tracer catalogue is generated. The cell size of the chain-mesh is equal to 2 times the value of the mean interparticle separation of the tracer catalogue. The maximum radius of the chain-mesh is given by the maximum value of the radius of the voids in the catalogue.

In [17]:
ChM = cbl.ChainMesh3D (2.*trcat.mps(),
                       trcat.var(cbl.Var__X_),
                       trcat.var(cbl.Var__Y_),
                       trcat.var(cbl.Var__Z_),
                       vdcat.Max(cbl.Var__Radius_))

If the void radii are not read from the input void catalogue, they are temporarily set to the maximum value of the range of the accepted radii:

In [18]:
if not param.findBool('Radius') :
  limit = param.findVectorDouble('delta_r')
  radii = [delta_r[1] for ii in range(vdcat.nObjects())]
  vdcat.set_var(cbl.Var__Radius_, radii)

The central density and the density contrast are computed if they are not read from the input void catalogue. <br> The central density (in units of the average density) is computed as the density of a sphere centred in the void centre and with radius $R = ratio \cdot r_{eff}$, where $r_{eff}$ is the void effective radius and $ratio$ is a parameter $<1$ selected by the user. The density contrast is the ratio between the central density and the density within the sphere centred in the void centre and with radius $R = r_{eff}$. <br> With the compute_densityContrast function, the effect of cloud-in-void is taken into account and the voids with central density larger than the density at $r_{eff}$ are rejected.

In [19]:
if not param.findBool('centralDensity') : 
  vdcat.compute_centralDensity(trcat,
                               ChM,
                               trcat.numdensity(),
                               param.findDouble('ratio'))

if not param.findBool('densityContrast') :
  vdcat.compute_densityContrast(trcat,
                                ChM,
                                param.findDouble('ratio'))

CBL > I removed 0 voids in calculating the central density!
CBL > Cloud-in-void: 1
CBL > I removed 1 voids in calculating the density contrast!


The criterion for the overlap-check is read from the parameter file. In case of overlap: <br>
1) if ol_crit = false $\rightarrow$ the void with the higher central density is rejected, <br>
2) if ol_crit = true $\rightarrow$ the void with the lower density constrast is rejected.

In [20]:
ol_crit = cbl.Var__DensityContrast_ if param.findInt('ol_crit') == 1 else cbl.Var__CentralDensity_

The threshold is the value of the spherically-averaged density contrast ($\rho_m/\,\overline{\rho}+1$) that each void will contain after the rescaling procedure. In order to obtain the value of the threshold at redshifts $z>0$, it is necessary to rescale it using the growth factor, computed with cosm.DD(z). This procedure has to be performed using the quantities in linear regime. In the end the threshold is converted back in the non-linear regime. 

In [21]:
Z0 = 0.
Z1 = param.findDouble('redshift')
threshold_L = cosm.deltav_L(1.,param.findDouble('threshold'))*cosm.DD(Z1)/cosm.DD(Z0)
threshold = 1. + cosm.deltav_NL(threshold_L)

# Costruct the cleanded catalogue
To build the final cleaned void catalogue, the user can select different procedures to perform using the following parameters: <br>
 - clean1 = true $\rightarrow$ erase voids with underdensities higher than a given threshold<br>
 - clean2 = true $\rightarrow$ erase voids with effective radii outside a given range  <br>
 - clean3 = true $\rightarrow$ erase voids with density contrast lower than a given value, specified by the 'relevance' parameter <br>
 - delta_r $\rightarrow$ range of acceptable radii, voids with radii outside this range are erased <br>
 - rescale = true $\rightarrow$ the rescaling procedure will be performed: firstly the algorithm checks if within an initial radius the enclosed density is higher or lower than the selected density threshold; then it shrinks or expands the initial radius to match the required density threshold <br>
 - overlap = true $\rightarrow$  overlapping voids are erased from the catalogue: when two voids do overlap one of them is erased according to the chosen overlap criterion.

In [23]:
print '\n'
tw0 = time.time()
tc0 = time.clock()
vdcat_cleaned = cbl.Catalogue (vdcat,
                               [param.findBool('clean1'),
                                param.findBool('clean2'),
                                param.findBool('clean3')],
                               param.findVectorDouble('delta_r'),
                               threshold,
                               param.findDouble('relevance'),
                               param.findBool('rescale'),
                               trcat,
                               ChM,
                               param.findDouble('ratio'),
                               param.findBool('overlap'),
                               ol_crit)


print 'Cleaning the catalogue took: ', time.clock()-tc0, ' sec'
print 'Wall time: ', time.time()-tw0, ' sec'



Cleaning the catalogue took:  0.431077  sec
Wall time:  0.431853055954  sec
CBL > Input void catalogue cleaning procedure started ...
CBL > ############ Removed Voids ###########
	 r_min - r_max criterion : 0
	 central density too high: 0
	 statistically irrelevant: 0
	 total removed: 0
CBL > 
 Voids in the Catalogue: 130
CBL > Time spent by the cleaning procedure: 6.9e-05 seconds 

CBL > Rescaling radii ... 
CBL > Sample volume = 999851 (Mpc/h)^3
CBL > Sample density = 0.0417122 (Mpc/h)^-3
CBL > Sample mps = 2.88345 Mpc/h
CBL > 	 Empty voids removed: 0
CBL > 	 Negative voids removed: 2

CBL > 	 Removed voids out of range [0.50,50.00] : 17
CBL > I removed 6 voids in calculating the central density!
CBL > Cloud-in-void: 0
CBL > I removed 5 voids in calculating the density contrast!
CBL > 
 Voids in the Catalogue: 100
CBL > Time spent by the rescaling procedure: 0.146309 seconds 

CBL > Checking for overlapping voids ...
CBL > I'm sorting the catalogue...
CBL > 	 Generating chain-mesh 

In the end, the cleaned catalogue is stored in an ASCII file (if the output directory does not exists, it will be created).

In [24]:
clmnsToPrint = cbl.VarCast(attrAv)

if not os.path.exists(param.findString('outputDir')):
    os.makedirs(param.findString('outputDir'))
    
print '\n'

vdcat_cleaned.write_data(param.findString('outputDir')+'/'+param.findString('outputFile'),
                         clmnsToPrint)



CBL > I'm writing the file: ../output/void_catalogue_cleaned.out...
CBL > I wrote the file: ../output/void_catalogue_cleaned.out
